<a href="https://colab.research.google.com/github/finiteautomata/sentiment-elecciones/blob/master/notebooks/Targeted_Sentiment_Analysis_BERT_Experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture 
!pip install datasets transformers ipdb

In [ ]:
from datasets import load_dataset, concatenate_datasets

dataset = load_dataset("finiteautomata/tsa")

dataset = concatenate_datasets(dataset.values())

Using custom data configuration finiteautomata--tsa-48c5e891fefc8aa8
Reusing dataset parquet (/root/.cache/huggingface/datasets/finiteautomata___parquet/finiteautomata--tsa-48c5e891fefc8aa8/0.0.0/0b6d5799bb726b24ad7fc7be720c170d8e497f575d02d47537de9a5bac074901)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

def split_dataset(dataset, test_size=0.2, random_state=None):
    df = dataset.to_pandas()
    id_noticias = df.groupby("id_noticia").count()["label"]

    train_ids, test_ids = train_test_split(id_noticias, test_size=test_size, random_state=random_state)
    train_ids, dev_ids = train_test_split(train_ids, test_size=test_size, random_state=random_state)

    df_train = df[df["id_noticia"].isin(train_ids.index)]
    df_dev = df[df["id_noticia"].isin(dev_ids.index)]
    df_test = df[df["id_noticia"].isin(test_ids.index)]
    
    columns = ["titulo", "id_noticia", "target", "label"]
    features = dataset.features

    train_dataset = Dataset.from_pandas(df_train[columns], features=features)
    dev_dataset = Dataset.from_pandas(df_dev[columns], features=features)
    test_dataset = Dataset.from_pandas(df_test[columns], features=features)

    return DatasetDict(train=train_dataset, dev=dev_dataset, test=test_dataset)



In [ ]:
id2label = dataset.features["label"].names

label2id = { k:i for i, k in enumerate(id2label)}

label2id

{'NEG': 0, 'NEU': 1, 'POS': 2}

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = 'dccuchile/bert-base-spanish-wwm-uncased'


tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.model_max_length = 128

loading configuration file https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/2416dab24674c27b5521594d6aa0929fc843a024c96711b1b5015cdff867291f.afa3630b664b4bd3e82d41660bdb96ec13236bbceadb0ae7c45c7c19f58652c7
Model config BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.4",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 31002


In [ ]:
def tokenize_with_target(example):
    return tokenizer(example["titulo"], example["target"], truncation=True)

def tokenize_without_target(example):
    return tokenizer(example["titulo"], truncation=True)

Perfecto, el segundo no tiene target

## Evaluación

In [ ]:
import torch
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report

def compute_metrics(pred):
    """
    Compute metrics for Trainer
    """
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)

    metrics = {}

    report = classification_report(labels, preds, output_dict=True)

    for i, cat in enumerate(id2label):
        rep = report[str(i)]
        metrics[cat.lower()+" F1"] = rep["f1-score"]
        metrics[cat.lower()+" Precision"] = rep["precision"]
        metrics[cat.lower()+" Recall"] = rep["recall"]

    metrics["macro_f1"] = report["macro avg"]["f1-score"]
    metrics["macro_precision"] = report["macro avg"]["precision"]
    metrics["macro_recall"] = report["macro avg"]["recall"]

    metrics["acc"] = report["accuracy"]

    return metrics

## Entrenamiento

In [ ]:
from transformers import TrainingArguments, Trainer, DataCollatorWithPadding

def run_experiments(dataset_dict, model_name, batch_size=32, epochs=5, warmup=0.1):
    model = AutoModelForSequenceClassification.from_pretrained(
        model_name, return_dict=True, num_labels=len(id2label)
    )
    total_steps = (epochs * len(dataset_dict["train"])) // batch_size
    warmup_steps = int(warmup * total_steps)


    training_args = TrainingArguments(
        output_dir='./results/',
        num_train_epochs=epochs,
        learning_rate=2e-5,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_steps=warmup_steps,
        do_eval=False,
        weight_decay=0.01,
        logging_dir='./logs',
        load_best_model_at_end=True,
        metric_for_best_model="macro_f1",
        evaluation_strategy="epoch",
        save_strategy="epoch",
        
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        compute_metrics=lambda x: compute_metrics(x),
        train_dataset=dataset_dict["train"],
        data_collator=DataCollatorWithPadding(tokenizer),
        eval_dataset=dataset_dict["dev"],
    )

    trainer.train()

    return trainer.evaluate(dataset_dict["test"])

## 10-fold Monte Carlo Validation

In [ ]:
import time
import random
from tqdm.auto import tqdm
from transformers import set_seed

folds = 20
target_results = []
no_target_results = []


titulares = []

for i in tqdm(range(folds)):
    seed = time.time()
    set_seed(int(seed))
    test_size = random.normalvariate(0.2, 0.02)
    print(f"Epoch no {i+1}")
    print(f"{test_size:.3f} test size" )

    
    ds_dict = split_dataset(dataset, test_size=test_size, random_state=int(seed))

    
    targeted_dataset = ds_dict.map(tokenize_with_target)
    not_targeted_dataset = ds_dict.map(tokenize_without_target)

    titulares_test = sorted({ex["titulo"] for ex in ds_dict["test"]})
    titulares.append(titulares_test)

    target_results.append(run_experiments(targeted_dataset, model_name))
    no_target_results.append(run_experiments(not_targeted_dataset, model_name))


  0%|          | 0/20 [00:00<?, ?it/s]

Epoch no 1
0.166 test size


  0%|          | 0/1697 [00:00<?, ?ex/s]

  0%|          | 0/339 [00:00<?, ?ex/s]

  0%|          | 0/403 [00:00<?, ?ex/s]

...

Epoch,Training Loss,Validation Loss,Neg f1,Neg precision,Neg recall,Neu f1,Neu precision,Neu recall,Pos f1,Pos precision,Pos recall,Macro F1,Macro Precision,Macro Recall,Acc
1,No log,0.869525,0.567686,0.550847,0.585586,0.515021,0.625000,0.437956,0.672727,0.609890,0.750000,0.585145,0.595246,0.591181,0.595960
2,No log,0.775218,0.684411,0.592105,0.810811,0.611354,0.760870,0.510949,0.713333,0.703947,0.722973,0.669699,0.685641,0.681578,0.674242
3,No log,0.820334,0.716981,0.616883,0.855856,0.540284,0.770270,0.416058,0.721519,0.678571,0.770270,0.659595,0.688575,0.680728,0.671717
4,No log,0.826234,0.716157,0.694915,0.738739,0.641026,0.773196,0.547445,0.741641,0.674033,0.824324,0.699608,0.714048,0.703503,0.704545
5,No log,0.810357,0.700461,0.716981,0.684685,0.634921,0.695652,0.583942,0.736842,0.680000,0.804054,0.690741,0.697544,0.690893,0.694444


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: titulo, target, id_noticia. If titulo, target, id_noticia are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 396
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-49
Configuration saved in ./results/checkpoint-49/config.json
Model weights saved in ./results/checkpoint-49/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: titulo, target, id_noticia. If titulo, target, id_noticia are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 396
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-98
Configuration saved in ./resul

loading configuration file https://huggingface.co/dccuchile/bert-base-spanish-wwm-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/2416dab24674c27b5521594d6aa0929fc843a024c96711b1b5015cdff867291f.afa3630b664b4bd3e82d41660bdb96ec13236bbceadb0ae7c45c7c19f58652c7
Model config BertConfig {
  "_name_or_path": "dccuchile/bert-base-spanish-wwm-uncased",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_toke

Epoch,Training Loss,Validation Loss,Neg f1,Neg precision,Neg recall,Neu f1,Neu precision,Neu recall,Pos f1,Pos precision,Pos recall,Macro F1,Macro Precision,Macro Recall,Acc
1,No log,0.923093,0.404494,0.537313,0.324324,0.526316,0.542636,0.510949,0.603448,0.525000,0.709459,0.511419,0.534983,0.514911,0.532828
2,No log,0.840347,0.607143,0.601770,0.612613,0.567901,0.650943,0.503650,0.670769,0.615819,0.736486,0.615271,0.622844,0.617583,0.621212
3,No log,0.876238,0.611321,0.525974,0.729730,0.571429,0.648148,0.510949,0.659574,0.694030,0.628378,0.614108,0.622717,0.623019,0.616162
4,No log,0.935322,0.604651,0.625000,0.585586,0.566667,0.660194,0.496350,0.682493,0.608466,0.777027,0.617937,0.631220,0.619654,0.626263
5,No log,0.938972,0.593607,0.601852,0.585586,0.604651,0.644628,0.569343,0.673016,0.634731,0.716216,0.623758,0.627070,0.623715,0.628788


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: titulo, target, id_noticia. If titulo, target, id_noticia are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 396
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-49
Configuration saved in ./results/checkpoint-49/config.json
Model weights saved in ./results/checkpoint-49/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: titulo, target, id_noticia. If titulo, target, id_noticia are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 396
  Batch size = 32
Saving model checkpoint to ./results/checkpoint-98
Configuration saved in ./resul

In [ ]:
import numpy as np
import scipy.stats
import pandas as pd

data = [
    *[{**{"target": "targeted"}, **res} for res in target_results],
    *[{**{"target": "none"}, **res} for res in no_target_results]
]

df_res = pd.DataFrame(data)


df_mean = df_res.groupby("target").mean().round(3)
df_std = df_res.groupby("target").std().round(3)

df_str_res = df_mean.astype(str) + " +- " + df_std.astype(str)

df_str_res

,eval_loss,eval_neg F1,eval_neg Precision,eval_neg Recall,eval_neu F1,eval_neu Precision,eval_neu Recall,eval_pos F1,eval_pos Precision,eval_pos Recall,eval_macro_f1,eval_macro_precision,eval_macro_recall,eval_acc,eval_runtime,eval_samples_per_second,eval_steps_per_second,epoch
target,,,,,,,,,,,,,,,,,,
none,0.925 +- 0.067,0.625 +- 0.028,0.604 +- 0.046,0.655 +- 0.065,0.589 +- 0.039,0.62 +- 0.045,0.565 +- 0.057,0.659 +- 0.029,0.662 +- 0.044,0.662 +- 0.056,0.624 +- 0.023,0.629 +- 0.02,0.627 +- 0.022,0.629 +- 0.022,0.632 +- 0.048,767.516 +- 15.768,24.696 +- 0.6,5.0 +- 0.0
targeted,0.85 +- 0.068,0.688 +- 0.035,0.679 +- 0.043,0.702 +- 0.056,0.596 +- 0.043,0.619 +- 0.055,0.578 +- 0.05,0.712 +- 0.03,0.705 +- 0.038,0.722 +- 0.046,0.666 +- 0.029,0.668 +- 0.027,0.667 +- 0.03,0.67 +- 0.029,0.678 +- 0.048,714.773 +- 18.067,23.0 +- 0.693,5.0 +- 0.0


In [ ]:
df_str_res.columns

Index(['eval_loss', 'eval_neg F1', 'eval_neg Precision', 'eval_neg Recall',
       'eval_neu F1', 'eval_neu Precision', 'eval_neu Recall', 'eval_pos F1',
       'eval_pos Precision', 'eval_pos Recall', 'eval_macro_f1',
       'eval_macro_precision', 'eval_macro_recall', 'eval_acc', 'eval_runtime',
       'eval_samples_per_second', 'eval_steps_per_second', 'epoch'],
      dtype='object')

In [ ]:
df_str_res[["eval_neg F1", "eval_neu F1", "eval_pos F1", "eval_macro_f1", "eval_acc"]]

,eval_neg F1,eval_neu F1,eval_pos F1,eval_macro_f1,eval_acc
target,,,,,
none,0.625 +- 0.028,0.589 +- 0.039,0.659 +- 0.029,0.624 +- 0.023,0.629 +- 0.022
targeted,0.688 +- 0.035,0.596 +- 0.043,0.712 +- 0.03,0.666 +- 0.029,0.67 +- 0.029


In [ ]:
scipy.stats.wilcoxon(
    x=[r["eval_macro_f1"] for r in target_results],
    y=[r["eval_macro_f1"] for r in no_target_results],
    alternative="greater"
)

WilcoxonResult(statistic=209.0, pvalue=5.167324782326882e-05)